In [7]:
import pandas as pd
from IPython.display import display
purchase_data = pd.read_csv("./Resources/purchase_data.csv")

In [212]:
unique_players = pd.DataFrame(purchase_data.SN.unique(), columns=['0'])
total_players = pd.DataFrame(unique_players.count(), columns=['Total Players'])
display(total_players)

,Total Players
0,576


In [214]:
total_purchases = pd.Series(pd.DataFrame(purchase_data['Purchase ID']).count()).rename(index={"Purchase ID":0})
unique_items = pd.Series(pd.DataFrame(purchase_data['Item ID'].unique()).count())
average_price = pd.Series(purchase_data['Price'].mean())
total_revenue = pd.Series(purchase_data['Price'].sum())

final_df = pd.concat([unique_items,average_price,total_purchases,total_revenue],axis=1,ignore_index=True).rename(columns={
    0: "Number of Unique Items",
    1: "Average Price",
    2: "Number of Purchases",
    3: "Total Revenue"})
final_df["Average Price"] = final_df["Average Price"].apply(lambda x: "${:.2f}".format((x)))
final_df["Total Revenue"] = final_df["Total Revenue"].apply(lambda x: "${:,.2f}".format((x)))
display(final_df)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [215]:
deduped_players = purchase_data.drop_duplicates(subset = ['SN'])
total_countm = pd.DataFrame(deduped_players.loc[deduped_players['Gender']=='Male']).count().iloc[0]
total_countf = pd.DataFrame(deduped_players.loc[deduped_players['Gender']=='Female']).count().iloc[0]
total_counto = pd.DataFrame(deduped_players.loc[deduped_players['Gender']=='Other / Non-Disclosed']).count().iloc[0]
total_count = pd.DataFrame(data=[total_countm, total_countf, total_counto], index=["Male","Female", "Other / Non-Disclosed"], columns=["Total Count"])
per_count = deduped_players['Gender'].value_counts(normalize=True) * 100
per_count = per_count.apply(lambda x: "{:,.2f}%".format((x)))
final_df = pd.concat([total_count, per_count], axis=1)
display(final_df)


,Total Count,Gender
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [216]:
per_count = purchase_data['Gender'].value_counts()
avg_purchases_by_gender = purchase_data.groupby('Gender')['Price'].mean().apply(lambda x: "${:.2f}".format((x)))
purchases_by_gender = purchase_data.groupby('Gender')['Price'].sum().apply(lambda x: "${:,.2f}".format((x)))
# This one isnt quite right, but I don't know what they are looking for
purchases_per_person_by_gender = purchase_data.groupby('Gender')['Price'].sum().divide(pd.to_numeric(per_count)).apply(lambda x: "${:,.2f}".format((x)))

final_df = pd.concat([per_count, avg_purchases_by_gender, purchases_by_gender, purchases_per_person_by_gender], axis=1)
final_df = final_df.rename(columns={final_df.columns[0]:'Purchase Count', final_df.columns[1]:'Average Purchase Price', final_df.columns[2]:'Total Purchase Value', final_df.columns[3]:'Avg Total Purchase per Person'})
display(final_df)

,Purchase Count,Total Purchase Value,Total Purchase Value,Avg Total Purchase per Person
Male,652,$3.02,"$1,967.64",$3.02
Female,113,$3.20,$361.94,$3.20
Other / Non-Disclosed,15,$3.35,$50.19,$3.35


In [218]:
cut_bins = [0, 9, 14, 19, 24, 29, 34, 39, 1000]
cut_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
binned_data = deduped_players.groupby([pd.cut(deduped_players['Age'], bins=cut_bins,labels=cut_labels)])
count_by_age = binned_data.size()
fraction_of_tot_by_age = count_by_age.divide(deduped_players.count().values[0]) * 100
percentage_by_age = fraction_of_tot_by_age.apply(lambda x: "{:.2f}%".format((x)))

final_df = pd.concat([count_by_age, percentage_by_age], axis=1)
final_df = final_df.rename(columns={final_df.columns[0]: "Total Count", final_df.columns[1]: "Percentage of Players"})
display(final_df)

,Total Count,Percentage of Players
Age,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%
